In [125]:
import random
from queue import PriorityQueue

## BFS Algorithm

In [126]:
def build_path(node):
    path = []
    while node:
        parent_node_number = node['parent']['node_number'] if node['parent'] else None
        path.append((node['state'], node['move'], node['node_number'], parent_node_number))
        node = node['parent']

    return path[::-1]

In [127]:
# Global counter for node numbers
node_counter = 0

def expand(node, board_side):
    global node_counter
    neighbors = []
    index = node['state'].index(0)
    moves = [('left', -1), ('up', -board_side), ('right', 1), ('down', board_side)]

    for m, offset in moves:
        new_index = index + offset
        if 0 <= new_index < len(node['state']) and (index % board_side != 0 or m != 'left') and ((index + 1) % board_side != 0 or m != 'right'):
            new_state = node['state'][:]
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            node_counter += 1
            neighbors.append({
                'state': new_state,
                'parent': node,
                'move': m,
                'depth': node['depth'] + 1,
                'node_number': node_counter
            })

    return neighbors



In [128]:
def bfs_solve(initial_state):
    global node_counter
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)
    explored, queue = set(), [{'state': initial_state, 'parent': None, 'move': None, 'depth': 0, 'node_number': 0}]
    node_counter = 0

    while queue:
        node = queue.pop(0)
        if node['state'] == goal_state:
            return build_path(node), node['depth']

        if tuple(node['state']) not in explored:
            explored.add(tuple(node['state']))
            neighbors = expand(node, board_side)
            queue.extend(neighbors)

    return None, None



## DFS Algorithm

In [129]:
def build_path_dfs(node):
    path = []
    while node:
        parent_node_number = node['parent']['node_number'] if node['parent'] else None
        path.append((node['state'], node['move'], node['node_number'], parent_node_number))
        node = node['parent']

    return path[::-1]

In [130]:
def expand_dfs(node, board_side):
    global node_counter
    neighbors = []
    index = node['state'].index(0)
    moves = [('left', -1), ('up', -board_side), ('right', 1), ('down', board_side)]

    for m, offset in moves:
        new_index = index + offset
        if 0 <= new_index < len(node['state']) and (index % board_side != 0 or m != 'left') and ((index + 1) % board_side != 0 or m != 'right'):
            new_state = node['state'][:]
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            node_counter += 1
            neighbors.append({
                'state': new_state,
                'parent': node,
                'move': m,
                'depth': node['depth'] + 1,
                'node_number': node_counter
            })

    return neighbors

In [131]:
def dfs_solve(initial_state):
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)

    node_counter = 0
    start_node = {
        'state': initial_state, 
        'parent': None, 
        'move': None, 
        'depth': 0, 
        'node_number': node_counter
    }

    stack = [start_node]  # Using a stack for DFS
    explored = set()

    while stack:
        current_node = stack.pop()
        current_state = current_node['state']

        if current_state == goal_state:
            return build_path_dfs(current_node), current_node['depth']

        if tuple(current_state) not in explored:
            explored.add(tuple(current_state))
            for neighbor in expand_dfs(current_node, board_side):
                if tuple(neighbor['state']) not in explored:
                    stack.append(neighbor)

    return None, None

## IDS Algorithm

In [132]:
def build_path_ids(node):
    path = []
    while node:
        parent_node_number = node['parent']['node_number'] if node['parent'] else None
        path.append((node['state'], node['move'], node['node_number'], parent_node_number))
        node = node['parent']

    return path[::-1]


In [133]:
def expand_ids(node, board_side):
    global node_counter
    neighbors = []
    index = node['state'].index(0)
    moves = [('left', -1), ('up', -board_side), ('right', 1), ('down', board_side)]

    for m, offset in moves:
        new_index = index + offset
        if 0 <= new_index < len(node['state']) and (index % board_side != 0 or m != 'left') and ((index + 1) % board_side != 0 or m != 'right'):
            new_state = node['state'][:]
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            node_counter += 1
            neighbors.append({
                'state': new_state,
                'parent': node,
                'move': m,
                'depth': node['depth'] + 1,
                'node_number': node_counter
            })

    return neighbors

In [134]:
def depth_limited_search(initial_state, goal_state, limit, board_side, node_counter):
    stack = [{'state': initial_state, 'parent': None, 'move': None, 'depth': 0, 'node_number': node_counter}]
    explored = set()

    while stack:
        node = stack.pop()
        if node['depth'] > limit:
            continue

        if node['state'] == goal_state:
            return build_path_ids(node), node['depth']

        explored.add(tuple(node['state']))
        for neighbor in expand_ids(node, board_side):
            if tuple(neighbor['state']) not in explored:
                stack.append(neighbor)

    return None, None


In [135]:
def ids_solve(initial_state):
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)

    node_counter = 0
    for depth_limit in range(1, 100):  # Assuming a reasonable depth limit
        result, depth = depth_limited_search(initial_state, goal_state, depth_limit, board_side, node_counter)
        if result is not None:
            return result, depth
    return None, None

## A* Algorithm

In [136]:
def manhattan_distance(state, goal_state):
    distance = 0
    side_length = int(len(state) ** 0.5)
    for num in range(1, len(state)):
        current_idx = state.index(num)
        goal_idx = goal_state.index(num)
        current_row, current_col = divmod(current_idx, side_length)
        goal_row, goal_col = divmod(goal_idx, side_length)
        distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance

In [137]:
def build_path_a_star(node):
    path = []
    while node:
        parent_node_number = node['parent']['node_number'] if node['parent'] else None
        path.append((node['state'], node['move'], node['node_number'], parent_node_number))
        node = node['parent']

    return path[::-1]

In [138]:
def expand_a_star(node, board_side):
    global node_counter
    neighbors = []
    index = node['state'].index(0)
    moves = [('left', -1), ('up', -board_side), ('right', 1), ('down', board_side)]

    for m, offset in moves:
        new_index = index + offset
        if 0 <= new_index < len(node['state']) and (index % board_side != 0 or m != 'left') and ((index + 1) % board_side != 0 or m != 'right'):
            new_state = node['state'][:]
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            node_counter += 1
            neighbors.append({
                'state': new_state,
                'parent': node,
                'move': m,
                'depth': node['depth'] + 1,
                'node_number': node_counter,
                'cost': 0
            })

    return neighbors


In [139]:
def a_star_solve(initial_state):
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)

    node_counter = 0
    start_node = {
        'state': initial_state, 
        'parent': None, 
        'move': None, 
        'depth': 0, 
        'node_number': node_counter, 
        'cost': manhattan_distance(initial_state, goal_state)
    }
    
    open_set = [start_node]
    explored = set()

    while open_set:
        # Sort the open set based on cost and pick the node with the lowest cost
        open_set.sort(key=lambda node: node['cost'])
        current_node = open_set.pop(0)
        current_state = current_node['state']

        if current_state == goal_state:
            return build_path_a_star(current_node), current_node['depth']

        explored.add(tuple(current_state))

        for neighbor in expand_a_star(current_node, board_side):
            if tuple(neighbor['state']) not in explored:
                neighbor['cost'] = neighbor['depth'] + manhattan_distance(neighbor['state'], goal_state)
                open_set.append(neighbor)

    return None, None

## IDA* Algorithm

In [140]:
def ida_search(node, g_cost, threshold, goal_state, board_side):
    f_cost = g_cost + manhattan_distance(node['state'], goal_state)
    if f_cost > threshold:
        return f_cost, None
    if node['state'] == goal_state:
        return f_cost, node

    min_cost = float('inf')
    for neighbor in expand_a_star(node, board_side):
        temp_cost, result = ida_search(neighbor, g_cost + 1, threshold, goal_state, board_side)
        if result is not None:
            return temp_cost, result
        if temp_cost < min_cost:
            min_cost = temp_cost

    return min_cost, None

In [141]:
def ida_star_solve(initial_state):
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)

    node_counter = 0
    start_node = {
        'state': initial_state, 
        'parent': None, 
        'move': None, 
        'depth': 0, 
        'node_number': node_counter, 
        'cost': manhattan_distance(initial_state, goal_state)
    }

    threshold = start_node['cost']
    while True:
        min_cost, result = ida_search(start_node, 0, threshold, goal_state, board_side)
        if result is not None:
            return build_path_a_star(result), result['depth']
        if min_cost == float('inf'):
            return None, None
        threshold = min_cost

## RBFS Algorithm

In [142]:
def rbfs_search(node, f_limit, goal_state, board_side):
    if node['state'] == goal_state:
        return node, node['f_cost']

    successors = expand_a_star(node, board_side)
    if not successors:
        return None, float('inf')

    for s in successors:
        s['f_cost'] = max(s['cost'] + manhattan_distance(s['state'], goal_state), node['f_cost'])

    while True:
        successors.sort(key=lambda x: x['f_cost'])
        best = successors[0]

        if best['f_cost'] > f_limit:
            return None, best['f_cost']

        alternative = successors[1]['f_cost'] if len(successors) > 1 else float('inf')
        result, best['f_cost'] = rbfs_search(best, min(f_limit, alternative), goal_state, board_side)

        if result is not None:
            return result, best['f_cost']

In [143]:
def rbfs_solve(initial_state):
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)

    node_counter = 0
    start_node = {
        'state': initial_state,
        'parent': None,
        'move': None,
        'depth': 0,
        'node_number': node_counter,
        'cost': manhattan_distance(initial_state, goal_state),
        'f_cost': manhattan_distance(initial_state, goal_state)
    }

    result, f_cost = rbfs_search(start_node, float('inf'), goal_state, board_side)
    if result is not None:
        return build_path_a_star(result), result['depth']
    return None, None



## Bidirectional Search

In [147]:
def build_path_bidirectional(node):
    path = []
    while node:
        parent_node_number = node['parent']['node_number'] if node['parent'] else None
        path.append((node['state'], node['move'], node['node_number'], parent_node_number))
        node = node['parent']

    return path[::-1]

In [148]:
def merge_paths(node_forward, node_backward):
    path_forward = build_path_bidirectional(node_forward)
    path_backward = build_path_bidirectional(node_backward)
    return path_forward[:-1] + path_backward[::-1]  # Remove duplicate meeting state and reverse backward path


In [149]:
def expand_bidirectional(node, board_side):
    global node_counter
    neighbors = []
    index = node['state'].index(0)
    moves = [('left', -1), ('up', -board_side), ('right', 1), ('down', board_side)]

    for m, offset in moves:
        new_index = index + offset
        if 0 <= new_index < len(node['state']) and (index % board_side != 0 or m != 'left') and ((index + 1) % board_side != 0 or m != 'right'):
            new_state = node['state'][:]
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            node_counter += 1
            neighbors.append({
                'state': new_state,
                'parent': node,
                'move': m,
                'depth': node['depth'] + 1,
                'node_number': node_counter
            })

    return neighbors

In [150]:
def bidirectional_search(initial_state):
    goal_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    board_side = int(len(initial_state) ** 0.5)

    node_counter = 0
    start_node = {'state': initial_state, 'parent': None, 'move': None, 'depth': 0, 'node_number': node_counter}
    goal_node = {'state': goal_state, 'parent': None, 'move': None, 'depth': 0, 'node_number': node_counter}

    if initial_state == goal_state:
        return build_path_bidirectional(start_node), 0

    frontier_forward = [start_node]
    frontier_backward = [goal_node]
    explored_forward = set()
    explored_backward = set()

    while frontier_forward and frontier_backward:
        # Forward search
        current_node_forward = frontier_forward.pop(0)
        explored_forward.add(tuple(current_node_forward['state']))

        # Backward search
        current_node_backward = frontier_backward.pop(0)
        explored_backward.add(tuple(current_node_backward['state']))

        # Check for meeting point
        if tuple(current_node_forward['state']) in explored_backward or tuple(current_node_backward['state']) in explored_forward:
            # Merge paths
            return merge_paths(current_node_forward, current_node_backward), current_node_forward['depth'] + current_node_backward['depth']

        # Expand nodes in forward direction
        for neighbor in expand_bidirectional(current_node_forward, board_side):
            if tuple(neighbor['state']) not in explored_forward:
                frontier_forward.append(neighbor)

        # Expand nodes in backward direction
        for neighbor in expand_bidirectional(current_node_backward, board_side):
            if tuple(neighbor['state']) not in explored_backward:
                frontier_backward.append(neighbor)

    return None, None


## Input Related Functions

In [151]:
def is_solvable(state):
    
    inversions = 0
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] > state[j] != 0:
                inversions += 1

    return inversions % 2 == 0



In [152]:
def move_tile(state, direction):
    new_state = list(state)
    index = new_state.index(0)
    
    if direction == 'up' and index >= 3:
        new_state[index], new_state[index - 3] = new_state[index - 3], new_state[index]
    elif direction == 'down' and index <= 5:
        new_state[index], new_state[index + 3] = new_state[index + 3], new_state[index]
    elif direction == 'left' and index % 3 != 0:
        new_state[index], new_state[index - 1] = new_state[index - 1], new_state[index]
    elif direction == 'right' and index % 3 != 2:
        new_state[index], new_state[index + 1] = new_state[index + 1], new_state[index]
    
    return new_state



In [153]:
def generate_random_puzzle(moves=100):
    state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    directions = ['up', 'down', 'left', 'right']
    for _ in range(moves):
        state = move_tile(state, random.choice(directions))
    return state


In [154]:
def get_initial_state_from_user():
    while True:
        try:
            # Ask the user to input the initial state
            user_input = input("Enter the initial state of the 8-puzzle as a list of numbers from 0 to 8 (e.g., 1 2 3 4 5 6 7 8 0), or type 'exit' to stop: ")
            if user_input.lower() == 'exit':
                print("Program exited by user.")
                exit()

            initial_state = [int(num) for num in user_input.split()]

            # Check if the input is valid
            if sorted(initial_state) == list(range(9)):
                return initial_state
            else:
                print("Invalid input. Please enter a list of numbers from 0 to 8 with no duplicates.")
        except ValueError:
            print("Invalid input. Please enter numbers only.")

### Main

In [155]:
# Using the generate_random_puzzle function to create an initial state
initial_state = generate_random_puzzle()


In [156]:

# Check if the initial state is solvable
if not is_solvable(initial_state):
    print("The initial state is not solvable.")
else:
    # Solve the 8-puzzle
    path, depth = sma_star_solve(initial_state)
    
    # Check if a solution was found
    if path:
        # Print the solution path, depth, and parent node numbers
        for state, move, node_number, parent_node_number in path:
            print(f'Node Number: {node_number}, Parent Node: {parent_node_number}')
            print('State:', state)
            print('Move:', move)
            print('*'*10)
        print('Depth:', depth)
    else:
        print("No solution found.")


TypeError: sma_star_solve() missing 1 required positional argument: 'max_nodes'